# Azure OpenAI - Function Calling

In [8]:
# Import required packages
import openai
import os
import datetime
import requests

# Define Azure OpenAI endpoint parameters
openai.api_type = "azure"
openai.api_version = "2023-05-15"
    # openai.api_version = "2023-03-15-preview"
openai.api_base = os.getenv("OPENAI_API_BASE") # Set AOAI endpoint value as env variable
openai.api_key = os.getenv("OPENAI_API_KEY") # Set AOAI API key as env variable
aoai_deployment = os.getenv("OPENAI_API_DEPLOY") # Set AOAI deployment name as env variable

# Define Alpha Vantage API parameter
av_api_key = os.getenv("ALPHAVANTAGE_API_KEY") # Set Alpha Vantage API key as env variable

In [17]:
# Get current date
today = datetime.date.today().strftime("%Y-%m-%d")
today

'2023-07-20'

In [21]:
# Function to get stock price from Alpha Vantage API
def get_stock_price(stock_symbol, stock_date):
    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_symbol}&apikey={av_api_key}"
        stock_data = requests.get(stock_url)
        stock_low = stock_data.json()["Time Series (Daily)"][stock_date]["3. low"]
        stock_high = stock_data.json()["Time Series (Daily)"][stock_date]["2. high"]
        return stock_low, stock_high
    except:
        return "NA", "NA"

In [22]:
# Testing stock price function
stock_low, stock_high = get_stock_price("IBM", today)
print(stock_low, stock_high)

NA NA


In [ ]:
functions = [
        {
            "name": "get_stock_price",
            "description": "Retrieve lowest and highest stock price for a given stock symbol and date",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "Stock symbol, for example MSFT for Microsoft"
                    },
                    "date": {
                        "type": "string",
                        "description": "Date in YYYY-MM-DD format"
                    }
                },
                "required": ["index"],
            },    
        }
    ]

available_functions = {
            "get_stock_price": get_stock_price,
        } 